In [1]:
from openai import OpenAI
import os
import requests
import json

In [2]:
from getpass import getpass



api_key = getpass("Enter your OpenAI API key: ")


os.environ["OPENAI_API_KEY"] = api_key


client = OpenAI(api_key=api_key)


In [11]:
tools=[{
    "type": "function",
    "function":{
        "name": "find_sources",
        "description":"Get sources for your research question",
        "parameters":{
            "type": "object",
            "properties":{
                "title":{"type":"string"}
        },
            "required":["title"],
            "additionalProperties": False
        },
        "strict":True
    }

}]

In [25]:
messages=[{"role":"user","content":"Find me an article on multiple regression "}]
def find_sources(title):
    response=requests.get("https://api.crossref.org/works"
        f"?query.title={title}"
        "&rows=5"
        "&filter=type:journal-article,type:proceedings-article")
    data=response.json()
    items = data.get("message", {}).get("items", [])
    results = []
    for item in items:
        title_list = item.get("title", [])
        if title_list:
            title_text = title_list[0]
            doi = item.get("DOI", "")
            results.append(f"{title_text} — https://doi.org/{doi}")

    return results 
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools
)

completion.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_Fsr1b3XWCJEjreirJv6VhD1H', function=Function(arguments='{"title":"multiple regression"}', name='find_sources'), type='function')]

In [26]:
tool_call=completion.choices[0].message.tool_calls[0]
args=json.loads(tool_call.function.arguments)

result=find_sources(args['title'])

messages.append(completion.choices[0].message)  
messages.append({                      
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": str(result)
})

completion_2 = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)
print(completion_2.choices[0].message.content)

Here are some articles on multiple regression:

1. [Multiple regression](https://doi.org/10.1136/bmj.f4373)
2. [Multiple Regression](https://doi.org/10.1055/s-2007-959036)
3. [Multiple Correlation Versus Multiple Regression](https://doi.org/10.1177/0013164402250990)
4. [Simple Regression in Multiple Regression Notation](https://doi.org/10.2307/2681496)
5. [Multiple monotone regression](https://doi.org/10.1037//0033-2909.92.3.791)

You can explore these articles for detailed information on multiple regression.
